In [1]:
import pandas as pd
import math
import plotly.express as px

# Cleaning Sample History

In [42]:
df1 = pd.read_csv("Sample-History1.csv")
df2 = pd.read_csv("Sample-History2.csv")
#df = df.rename(columns = {"Title": "History"})

In [43]:
df2

,Title,Date
0,Single’s Inferno: Season 2: Episode 2,2/20/23
1,Single’s Inferno: Season 2: Episode 1,1/12/23
2,Men on a Mission: 2022: 2022-02-12,1/7/23
3,Ronny Chieng: Speakeasy,12/10/22
4,Hasan Minhaj: The King's Jester,12/10/22
...,...,...
221,The Flash: Season 1: Fastest Man Alive,10/18/15
222,The Flash: Season 1: Pilot,10/18/15
223,Marvel's Daredevil: Rabbit in a Snowstorm,10/17/15
224,Marvel's Daredevil: Cut Man,7/6/15


In [44]:
# df['Date'] = pd.to_datetime(df['Date'])
# df['Day']= df['Date'].dt.day
# df['Month']= df['Date'].dt.month
# df['Year']= df['Date'].dt.year
# df['Day_of_week'] = df['Date'].dt.dayofweek # monday = 0, sunday = 6

In [45]:
# df

In [46]:
# df['Title'] = df['History'].str.rsplit(': ', 2).str[0]
# df['Season'] = df['History'].str.rsplit(': ', 2).str[1]
# df['Episode'] = df['History'].str.rsplit(': ', 2).str[2]

In [47]:
# df['Type'] = df['Episode'].apply(lambda x : 'Movie' if (pd.isna(x)==True) else 'TV')

Right now, movies that have a colon in their title have an incomplete value in the `Title`. I correct that below:

In [48]:
# tv = df[df['Type']!='Movie']
# movies = df[df['Type']=='Movie']
# movies['Title'] = movies['History']
# movies['Season'] = None

In [49]:
# tv.head()

In [50]:
# tv['Season'] = tv['Season'].str.split().str[1]

In [51]:
# df = pd.concat([movies, tv], ignore_index = True)

In [52]:
def clean_watch_history(df):
    '''
    Function that cleans a given users watch history data
    Input: dataframe
    Output: (cleaned) dataframe
    '''
    df = df.rename(columns = {"Title": "History"})
    df['Date'] = pd.to_datetime(df['Date'])
    df['Day']= df['Date'].dt.day
    df['Month']= df['Date'].dt.month
    df['Year']= df['Date'].dt.year
    df['Day_of_week'] = df['Date'].dt.dayofweek

    df['Title'] = df['History'].str.rsplit(': ', 2).str[0]
    df['Season'] = df['History'].str.rsplit(': ', 2).str[1]
    df['Episode'] = df['History'].str.rsplit(': ', 2).str[2]

    df['Type'] = df['Episode'].apply(lambda x : 'Movie' if (pd.isna(x)==True) else 'TV')

    tv = df[df['Type']!='Movie']
    tv['Season'] = tv['Season'].str.split().str[1]

    movies = df[df['Type']=='Movie']
    movies['Title'] = movies['History']
    movies['Season'] = None

    df = pd.concat([movies, tv], ignore_index = True)
    return df

In [53]:
df1 = clean_watch_history(df1)
df2 = clean_watch_history(df2)

/var/folders/nx/02f7zqm17f5_3khk_h9y42s00000gn/T/ipykernel_84740/3536717928.py:14: FutureWarning:

In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.

/var/folders/nx/02f7zqm17f5_3khk_h9y42s00000gn/T/ipykernel_84740/3536717928.py:15: FutureWarning:

In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.

/var/folders/nx/02f7zqm17f5_3khk_h9y42s00000gn/T/ipykernel_84740/3536717928.py:16: FutureWarning:

In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.

/var/folders/nx/02f7zqm17f5_3khk_h9y42s00000gn/T/ipykernel_84740/3536717928.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [54]:
df2[df2['History']=='Extraordinary Attorney Woo: The Blue Night of Jeju I']

,History,Date,Day,Month,Year,Day_of_week,Title,Season,Episode,Type
3,Extraordinary Attorney Woo: The Blue Night of ...,2022-12-03,3,12,2022,5,Extraordinary Attorney Woo: The Blue Night of ...,None,NaN,Movie


Issues:

1. (**SOLVED**) TV shows that have a colon in their title need to be handled differently. See `Avatar: The Last Airbender`.
    - Should we consider only calling split() on ': Season', ': Series', ': Book', etc? An obvious issue is it would be hard to compile an exhaustive such list. Other examples include Volume, Chapter, etc.
    - Alternatively, we could keep a list of all TV shows from the Netflix dataset that do have a colon. If any show then appears in that list, we could then make sure its first colon is not counted in the above procedure. Again, an obvious issue is figuring out how to determine that a given show is in that list (since this has to occur before we do any cleaning)
    - A potential solution is to do split on colon occurences in reverse order. For instance, only split based on the last two occurences of the colon character, and leave earlier colon occurences alone.
    
2. Movie titles may not exactly match the Netflix Dataset. See `Major (Telugu)`, which appears in the Netflix dataset just as `Major`, but in 3 different rows with 3 different languages.
    - One solution would be to remove the `([language])` substring in the sample dataset, and drop duplicates in the Netflix dataset (since the only differences between the duplicates would be the language).
    - Another solution would be to instead fuzzy match based on string similarity. Theoretically, this should work since exact matches would just have a 100% score. Plus, if titles in the Netflix dataset have this issue too (i saw at least one occurence), this might be able to handle those edge cases. Based on past experience, can be pain to implement (that being said, the logic is simpler than what I've previously had to deal with).
        - We could also think about first doing exact string matching, and then performing fuzzy matching on the remaining titles that haven't been matched.
    - Another solution is to just ignore :) and hope it doesn't affect our recommendations much
3. TV Shows that have a colon in their episode name. See `New Girl: Season 6: Operation: Bobcat`.
    - Similar issue to 1, but flipped. Hard to solve both issues 1 and 3. Maybe just ignore?

# Merging with Netflix Data

In [55]:
def netflix_merge(df):
    '''
    Function that merges given watch history with netflix dataset,
    and returns merged dataset
    '''
    titles = pd.read_csv('titles.csv')
    merged = df.merge(titles, left_on = 'Title', right_on = 'title', how = 'inner')
    cols_to_drop = ['production_countries', 'imdb_id', 'age_certification', 
                    'title', 'seasons', 'tmdb_popularity']
    merged = merged.drop(cols_to_drop, axis = 1)
    return merged

In [56]:
df1 = netflix_merge(df1)
df2 = netflix_merge(df2)

In [17]:
# titles = pd.read_csv("titles.csv")

In [ ]:
# merged = df.merge(titles, left_on = 'Title', right_on = 'title', how = 'inner')
# only 7 rows were dropped

In [ ]:
# merged.info()

In [ ]:
# cols_to_drop = ['type', 'production_countries', 'imdb_id', 'age_certification', 'id', 'title', 'seasons',
#                'tmdb_popularity']
# merged = merged.drop(cols_to_drop, axis = 1)

In [ ]:
# merged.head()

# Overlap Merge

In [ ]:
def overlap_merge(df1, df2):
    dfNew = df1.merge(df2, how = 'inner', left_on = 'History', right_on = 'History', suffixes=('', '_y'))
    dfNew = dfNew.drop(dfNew.filter(regex='_y$').columns, axis=1)
    return dfNew

In [ ]:
overlap = overlap_merge(df1, df2)

In [ ]:
overlap

# Aggregation

Stats/Visualizations to create:
- Individual
    - Viewing history by day of week
    - Top 10 Shows Watched
    - Movies most rewatched
    - Viewing history by month
    - Viewing history trend over time
    - Average watch time per day
- Common
    - Most common TV shows
    - Most common movies
    - Most common genres
    - Favorite actors/directors
    - Match score
- Overall
    - Minutes spent vs all other users
    - Percentile for top shows

**For all of these, we could use number of minutes watched (using the runtime column) rather than just the number of episodes**

1. Total Minutes Watched

In [ ]:
def total_minutes(df):
    fig = px.pie(df, values='runtime', names='Type', hole=.5)
    fig.update_traces(hovertemplate='Total Watch Time: %{value} mins')
    return fig

In [ ]:
total_minutes(df1).show()

In [ ]:
df = merged.groupby('Type').sum()

In [ ]:
fig = px.pie(merged, values='runtime', names='Type', hole=.5)
fig.update_traces(hovertemplate='Total Watch Time: %{value} mins')
fig.show()

2. Most watched TV shows

In [ ]:
def most_watched_tv(df):
    df = df[df['Type']=='TV']
    df = df.groupby('Title').sum()
    df = df.reset_index()
    
    df = df.sort_values(by='runtime', ascending= False)[:10]
    
    fig = px.bar(data_frame= df,
             x= "runtime",
             y= "Title",
             # text= "num_shared_actors",
             labels= {"runtime": "Minutes Watched"},
             text_auto= True)
    fig.update_traces(hovertemplate='Total Watch Time: %{x} mins')
    return fig, df[['Title', 'runtime']]

In [ ]:
fig, top_titles1 = most_watched_tv(df1)
fig, top_titles2 = most_watched_tv(df2)

In [ ]:
top_titles1

In [ ]:
df = merged[merged['Type']=='TV']
df = df.groupby('Title').sum()
df = df.reset_index()

In [ ]:
df = df.sort_values(by='runtime', ascending= False)
# extract 10 most similar shows
df = df.head(10)

In [ ]:
fig = px.bar(data_frame= df,
             x= "runtime",
             y= "Title",
             # text= "num_shared_actors",
             labels= {"runtime": "Minutes Watched"},
             text_auto= True)
fig.update_traces(hovertemplate='Total Watch Time: %{x} mins')
fig.show()

3. Most watched genre

In [ ]:
def most_watched_genre(df):
    df['genres'] = df['genres'].apply(lambda x: x.replace('\'', ''))
    df['genres'] = df['genres'].apply(lambda x: x.replace('[', ''))
    df['genres'] = df['genres'].apply(lambda x: x.replace(']', ''))
    df['genres'] = df['genres'].apply(lambda x: x.replace(' ', ''))

In [ ]:
df1

In [ ]:
df = df1
df['genres'] = df['genres'].apply(lambda x: x.replace('\'', ''))
df['genres'] = df['genres'].apply(lambda x: x.replace('[', ''))
df['genres'] = df['genres'].apply(lambda x: x.replace(']', ''))
df['genres'] = df['genres'].apply(lambda x: x.replace(' ', ''))

In [ ]:
df = df['genres'].str.split(',', expand = True)

In [ ]:
temp1 = df1['genres'].str.split(',', expand = True)
genre_col_list = ['genre_' + str(i) for i in range(len(temp1.columns))]
temp1.columns = genre_col_list

In [ ]:
genre_col_list


In [ ]:
temp = pd.concat([df1, temp1], axis = 1)

In [ ]:
s = pd.Series()
for i in genre_col_list:
    s = s.add(temp.groupby(i).sum()['runtime'], fill_value=0)

In [ ]:
s.idxmax()

In [ ]:
temp = pd.DataFrame(s.reset_index())
temp

In [ ]:
list(temp.sort_values(by=0, ascending=False)[:3]['index'])

In [20]:
def most_watched_genre(df):
    df['genres'] = df['genres'].apply(lambda x: x.replace('\'', ''))
    df['genres'] = df['genres'].apply(lambda x: x.replace('[', ''))
    df['genres'] = df['genres'].apply(lambda x: x.replace(']', ''))
    df['genres'] = df['genres'].apply(lambda x: x.replace(' ', ''))

    temp = df['genres'].str.split(',', expand = True)
    genre_col_list = ['genre_' + str(i) for i in range(len(temp.columns))]
    temp.columns = genre_col_list

    df = pd.concat([df, temp], axis = 1)

    s = pd.Series()
    for i in genre_col_list:
        s = s.add(df.groupby(i).sum()['runtime'], fill_value=0)
    
    df = pd.DataFrame(s.reset_index())
    top = df.sort_values(by=0, ascending=False)[:10]

    fig = px.pie(df, values=0, names='index', hole=.5)
    fig.update_traces(hovertemplate='Number of Minutes Watched: %{value}')
    return fig, top

In [21]:
fig, top_genre1 = most_watched_genre(df1)
fig, top_genre2 = most_watched_genre(df2)

/var/folders/nx/02f7zqm17f5_3khk_h9y42s00000gn/T/ipykernel_84740/2160281187.py:13: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

/var/folders/nx/02f7zqm17f5_3khk_h9y42s00000gn/T/ipykernel_84740/2160281187.py:15: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

/var/folders/nx/02f7zqm17f5_3khk_h9y42s00000gn/T/ipykernel_84740/2160281187.py:15: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

/var/folders/nx/02f7zqm17f5_3khk_h9y42s00000gn/T/ipykernel_84740/2160281187.py:15: FutureWarning:

The default v

In [31]:
top_genre1

,index,0
4,drama,6041.0
3,crime,4998.0
8,romance,3773.0
2,comedy,3399.0
10,thriller,2811.0
7,history,558.0
0,action,291.0
1,animation,24.0
5,family,24.0
6,fantasy,24.0


In [30]:
on = top_genre1.columns[0]

In [26]:
val = str(top_genre1.columns[1])
val_x = val+'_x'
val_y = val+'_y'

'0'

In [ ]:
df_genre = pd.merge(top_genre1, top_genre2, how='inner', on=on)
df_genre['score'] = (df_genre['0_x'] + df_genre['0_y'])/2
df_genre.sort_values(by='score', ascending=False)

In [59]:
def overlap(df1, df2):
    on = df1.columns[0]
    val = str(df1.columns[1])
    val_x = val+'_x'
    val_y = val+'_y'

    df = pd.merge(df1, df2, how='inner', on=on)
    df['score'] = (df[val_x] + df[val_y])/2
    df = df.sort_values(by='score', ascending=False)
    return list(df[on])

In [66]:
overlap(top_actors1, top_actors2)

['Hannah Simone',
 'Poorna Jagannathan',
 'Richa Shukla Moorjani',
 'Ramona Young',
 'Jaren Lewison',
 'John McEnroe',
 'Maitreyi Ramakrishnan',
 'Lee Rodriguez',
 'Darren Barnet',
 'Lee Jung-jae',
 'Kim Joo-ryoung',
 'Heo Sung-tae',
 'Jung Ho-yeon',
 'Wi Ha-jun',
 'Park Hae-soo',
 'Lee You-mi',
 'Oh Young-soo',
 'Anupam Tripathi',
 'Vir Das']

In [69]:
top_actors1[top_actors1['name']=="Lee Jung-jae"]

,name,Title
52,Lee Jung-jae,3


4. Top cast

In [ ]:
cast = pd.read_csv("credits.csv")

In [ ]:
cast = cast[cast['role']=='ACTOR']

In [ ]:
temp = df1.merge(cast, how='left', left_on='id', right_on='id')

In [ ]:
temp[temp['name']=='Poorna Jagannathan']

In [ ]:
temp = temp.groupby('name').count()['Title'].reset_index()

In [ ]:
temp.head()

In [ ]:
temp = temp.sort_values(by='Title', ascending= False)
# extract 10 most similar shows
temp = temp.head(10)
fig = px.bar(data_frame= temp,
             x= "Title",
             y= "name",
             # text= "num_shared_actors",
             labels= {"Title": "Number of Appearances"},
             text_auto= True)
fig.update_traces(hovertemplate='Total Appearances: %{x}')
fig.show()

In [62]:
def top_actors(df):
    cast = pd.read_csv("credits.csv")
    cast = cast[cast['role']=='ACTOR']
    df = df.merge(cast, how='left', left_on='id', right_on='id')

    df = df.groupby('name').count()['Title'].reset_index()
    df = df.sort_values(by='Title', ascending= False)

    fig = px.bar(data_frame= df[:10],
                x= "Title",
                y= "name",
                # text= "num_shared_actors",
                labels= {"Title": "Number of Appearances"},
                text_auto= True)
    fig.show()
    return df

In [63]:
top_actors1 = top_actors(df1)
top_actors2 = top_actors(df2)

5. language breakdown

In [ ]:
df1